In [ ]:
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline

In [2]:
books_cleaned_url = "https://raw.githubusercontent.com/jhlopesalves/classic_workflows/refs/heads/main/LLM/book_recommender/data/books_with_categories.csv"
books = pd.read_csv(books_cleaned_url)

## Emotion-Aware Signal for Recommendation

In addition to semantic similarity, recommendation systems can benefit from understanding the **emotional tone** of textual descriptions.

Book summaries often encode affective information — whether a narrative is melancholic, hopeful, tense, disturbing, or emotionally neutral — even when the topic or genre overlaps. Capturing this signal allows the system to distinguish between books that are semantically related but experientially different.


**Model:** `j-hartmann/emotion-english-distilroberta-base` <br>
**Source:** [https://huggingface.co/j-hartmann/emotion-english-distilroberta-base](https://huggingface.co/j-hartmann/emotion-english-distilroberta-base)

The model predicts a probability distribution over core emotional categories such as *joy, sadness, anger, fear, surprise,* and *neutral*, providing a compact emotional fingerprint for each description.


In [ ]:
classifier = pipeline(
	"text-classification", model="j-hartmann/emotion-english-roberta-large", top_k=None, device=0
)
classifier("I love this bloody thing, truly!")

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.5284838080406189},
  {'label': 'anger', 'score': 0.3548078238964081},
  {'label': 'disgust', 'score': 0.0656430795788765},
  {'label': 'neutral', 'score': 0.021273812279105186},
  {'label': 'sadness', 'score': 0.013662921264767647},
  {'label': 'surprise', 'score': 0.010343440808355808},
  {'label': 'fear', 'score': 0.005785102024674416}]]

## Classification on entire description

In [ ]:
display(f"Book Description: {books['description'][0]}\n")
print("\nLabels and Scores (entire description at once): ")
for pred in classifier(books["description"][0]):
	display(pred)

'Book Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the 


Labels and Scores (entire description at once): 


[{'label': 'fear', 'score': 0.43260717391967773},
 {'label': 'sadness', 'score': 0.22704674303531647},
 {'label': 'anger', 'score': 0.15658491849899292},
 {'label': 'neutral', 'score': 0.10718520730733871},
 {'label': 'disgust', 'score': 0.028648976236581802},
 {'label': 'surprise', 'score': 0.02760341390967369},
 {'label': 'joy', 'score': 0.02032361924648285}]

## Classification of single sentences

In [5]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [ ]:
display(f"Sentence: {sentences[0]}")
print("\nLabels and Scores (for a single sentence): ")
for pred in predictions[0]:
	print(pred)

'Sentence: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'


Labels and Scores (for a single sentence): 
{'label': 'joy', 'score': 0.37398281693458557}
{'label': 'surprise', 'score': 0.32151535153388977}
{'label': 'neutral', 'score': 0.2802889943122864}
{'label': 'fear', 'score': 0.012431065551936626}
{'label': 'anger', 'score': 0.00507175549864769}
{'label': 'sadness', 'score': 0.00441461568698287}
{'label': 'disgust', 'score': 0.0022953746374696493}


In [ ]:
display(f"Sentence: {sentences[3]}")
print("\nLabels and Scores (for a single sentence): ")
for pred in predictions[3]:
	print(pred)

'Sentence:  Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'


Labels and Scores (for a single sentence): 
{'label': 'anger', 'score': 0.9640884399414062}
{'label': 'neutral', 'score': 0.014637577347457409}
{'label': 'sadness', 'score': 0.010531258769333363}
{'label': 'disgust', 'score': 0.007427838630974293}
{'label': 'fear', 'score': 0.001942915259860456}
{'label': 'surprise', 'score': 0.0008065457222983241}
{'label': 'joy', 'score': 0.0005654493579640985}


In [ ]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


def calculate_max_emotion_scores(predictions):
	max_scores = {label: 0.0 for label in emotion_labels}
	for prediction in predictions:
		for item in prediction:
			label = item["label"]
			score = item["score"]
			if label in max_scores:
				max_scores[label] = max(max_scores[label], score)
	return max_scores

In [ ]:
# Pre-pull columns once (huge win vs books["col"][i] repeatedly)
isbn_col = books["isbn13"].tolist()
desc_col = books["description"].fillna("").astype(str).tolist()

# Prebind list append methods
score_append = {label: emotion_scores[label].append for label in emotion_labels}
isbn_append = isbn.append

for book_isbn, desc in tqdm(zip(isbn_col, desc_col), total=len(desc_col)):
	isbn_append(book_isbn)

	# strip + drop empties
	sentences = [s.strip() for s in desc.split(".") if s.strip()]

	if not sentences:
		# No text → all zeros (or change this behaviour if you prefer NaNs)
		max_scores = {label: 0.0 for label in emotion_labels}
	else:
		# Key speed knobs: batch_size + truncation/max_length
		predictions = classifier(
			sentences,
			top_k=None,
			batch_size=64,
			truncation=True,
			max_length=256,
		)
		max_scores = calculate_max_emotion_scores(predictions)

	for label in emotion_labels:
		score_append[label](max_scores[label])

  0%|          | 0/5197 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

books = pd.merge(books, emotions_df, on="isbn13")

In [14]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.100047,0.023109,0.827472,0.972007,0.956472,0.321515,0.914972
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.124159,0.254013,0.944703,0.090336,0.342887,0.333470,0.901248
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.427210,0.041528,0.801989,0.085785,0.022291,0.028694,0.102332
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.756360,0.025775,0.114194,0.043263,0.022732,0.014966,0.934166
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Fiction,0.186609,0.178644,0.123251,0.056377,0.332100,0.098040,0.886212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Nonfiction,0.734110,0.187842,0.842375,0.285075,0.473367,0.031734,0.727764
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.111409,0.110555,0.019815,0.663279,0.011627,0.124983,0.970020
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.032163,0.025610,0.575627,0.953516,0.095367,0.026217,0.176045
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.044385,0.078747,0.967261,0.651837,0.029512,0.021813,0.960081


In [15]:
books.to_csv("books_with_emotions.csv", index=False)